# IMPLICIT MODELING TUTORIAL
We introduce the opengeode implicit modeling tool. We will build a scalar field representaing stratigraphic horizons inside fault blocks. The workflow illustrated here follows 5 steps:
 1. Identify fault blocks by creating a geode structural model that only integrates faults. This first step requieres fault geometries to process with geode's explicit tools (repair and remesh).
 1. Build the horizon stack which describe the logical order of startigraphic dicontinuities. The horizon stack define the horizons and discontinuities that will be considered in the scalar field. Isovalues associated to each horizon is then implicitly sected.
 1. Integrate stratigraphic evidence, i.e. point coordinates where stratigraphic discontinuities have been observed. This step link points of the 3D space to their stratigraphic discontinuity in the horizon stack.
 1. Interpolate implicit function 
 1. Explicit horizons in the structural model 

In [ ]:
import opengeode as og 
import opengeode_geosciences as ogg
import geode_common as common
import geode_simplex as simplex
import geode_numerics
import geode_implicit 
import geode_explicit


# define various utilities functions
import os
def input(path):
  return os.path.join("data/",path) 

def create_folder_if_does_not_exists(path):
  if( not os.path.isdir(path)):
    os.makedirs(path)

def output(path):
  result_path = "results/"
  create_folder_if_does_not_exists(result_path)
  return os.path.join(result_path,path)

def create_aligned_box3D(extent_x,extent_y,extent_z):
    bbox = og.BoundingBox3D()
    bbox.add_point(og.Point3D([0,0,0]))
    bbox.add_point(og.Point3D([extent_x,extent_y,extent_z]))
    return bbox

print("Geode module imported!")

## Implicit modeling in a simple cube
### Create a cubic fault block 
steps:
 1. create six triangulated surfaces bounding a cube
 1. create a Brep using the repair technology in explicit modeling (no faults are considered in this exmaple then the cude is fault block)
 1. create a structural model from the brep

In [ ]:
# define function
def create_surface_from_orbital_points(points):
    surface = og.PolygonalSurface3D.create()
    builder = og.PolygonalSurfaceBuilder3D.create(surface)
    vertices=[]
    for point in points:
        vertices.append(builder.create_point(point))
    builder.create_polygon(vertices)
    og.triangulate_surface_mesh3D(surface)
    return og.convert_surface_mesh_into_triangulated_surface3D(surface)

# Create a cubic fault block
box = create_aligned_box3D(40, 40, 40)

# corner points on the top of the box. 
top1 = og.Point3D([box.min().value(0),box.min().value(1),box.max().value(2)])
top2 = og.Point3D([box.min().value(0),box.max().value(1),box.max().value(2)])
top3 = og.Point3D([box.max().value(0),box.max().value(1),box.max().value(2)])
top4 = og.Point3D([box.max().value(0),box.min().value(1),box.max().value(2)])
# corner points on the bottom of the box. 
bot1 = og.Point3D([box.min().value(0),box.min().value(1),box.min().value(2)])
bot2 = og.Point3D([box.min().value(0),box.max().value(1),box.min().value(2)])
bot3 = og.Point3D([box.max().value(0),box.max().value(1),box.min().value(2)])
bot4 = og.Point3D([box.max().value(0),box.min().value(1),box.min().value(2)])

# surface of the bounding box. 
top = create_surface_from_orbital_points([top1,top2,top3,top4])
bottom = create_surface_from_orbital_points([bot1,bot2,bot3,bot4])
front = create_surface_from_orbital_points([top1,top4,bot4,bot1])
back = create_surface_from_orbital_points([top2,top3,bot3,bot2])
left = create_surface_from_orbital_points([top1,top2,bot2,bot1])
right = create_surface_from_orbital_points([top3,top4,bot4,bot3])

# explicit modeling to create the BRep
modeler = geode_explicit.BRepExplicitModeler()
modeler.add_triangulated_surface(top)
modeler.add_triangulated_surface(bottom)
modeler.add_triangulated_surface(front)
modeler.add_triangulated_surface(back)
modeler.add_triangulated_surface(left)
modeler.add_triangulated_surface(right)
box_brep = modeler.build()

# mesh blocks
edge_length = 2
cst_metric = common.ConstantMetric3D(edge_length)
remeshed_brep = simplex.brep_simplex_remesh(box_brep,cst_metric)[0]

# build a structural model from the BRep
box_structural_model = ogg.StructuralModel(remeshed_brep)

#save to vtm for a visualization in paraview
og.save_brep(box_structural_model,output("box_brep.vtm"))

![Box structural model](./pictures/box_brep.png)

___
In short, INPUTS from feflow are:

 * fault surfaces --> Group of polygons in feflow tagged as a "fault" (Geode's need to know each fault)
 * boundary surfaces --> Group of polygons in feflow tagged as a "boundary" (Geode's nbeed to know each boundary to seal the model)
___

### Build the Horizons Stack
Here, we define the horizons stack by naming and organizing hozions and stratigraphic units. 

In [ ]:
# naming from the bottom to the top horizon
horizons_names = [ "hor_bot", "hor_top" ]
# naming from the bottom to the top stratigraphic unit 
stratigraphic_unit_names = [ "su_bot", "su_mid", "su_top" ]
# create horizons stacks from names
horizon_stack = ogg.horizons_stack_from_name_list_3d( horizons_names, stratigraphic_unit_names )

The horizons stacks requieres the name of each stratigraphic discontinuity in a "logical". A stratigraphic unit is the volume defined by two stratigraphic discontinuity (horizon). Geode can also store information about statigraphic units even if it is not requiered from the implicit modeling (names can be set by default from horizons' name).
___
In short, INPUTS from feflow are:

 * A list of names of each stratigrahic discontinuity (hrorizon/unconformity) to be considered.
 * The "logical" sequence of statigraphic discontinuities.
 * The type of each discontinuity (list to be discuss):
   * conformal
   * unconformal
   * erosion
   * baselap
___
Then the horizon stack is combinned with the structural model containing the definition of fault blocks.

In [ ]:
impliciter = geode_implicit.StructuralModelImpliciter( box_structural_model, horizon_stack)


### Intergrate stratigraphic evidences
The link between stratigraphic evidence in the 3D space (points) and the stratigraphic discontinuity defined in the horizons stack is built.
Here, we start by creating a point set for each horizon.

In [ ]:
# Point set Building 

#create data points
def create_points_set(points):
    mesh = og.PointSet3D.create()
    mesh_builder = og.PointSetBuilder3D.create(mesh)
    mesh_builder.create_vertices(len(points))
    id=0
    for pt in points:
        mesh_builder.set_point(id,pt)
        id+=1
    return mesh

# create horizon evidence  for bottom 
data_hor_bot = create_points_set([
    og.Point3D([5., 6.25, 9.5]),og.Point3D([29.5, 30.3, 7]),
    og.Point3D([12.1, 24.9, 9.5]), og.Point3D([27.3, 17.9, 7.5]),
    og.Point3D([14, 14.6, 9.]), og.Point3D([17, 21.95, 8]), 
    og.Point3D([22.14, 14.22, 5]), og.Point3D([17.2, 5.5, 10.5]), 
    og.Point3D([26.6, 9.27, 9.75]),og.Point3D([23.9, 24.5, 8.5]) ])

# save in vtp to be visualized in paraview
og.save_point_set3D( data_hor_bot, output("data_hor_bot.vtp") )

# create horizon evidence  for bottom 
data_hor_top = create_points_set([
    og.Point3D([8.6, 27.2, 25.5]), og.Point3D([13.6, 15., 24]), 
    og.Point3D([13.7, 6.55, 23.5]), og.Point3D([23.1, 26.98, 27.5]),
    og.Point3D([24.1, 10.2, 25]), og.Point3D([16.3, 25.7, 26]), 
    og.Point3D([35.1, 34.9, 26.5]) ])

# save in vtp to be visualized in paraview
og.save_point_set3D( data_hor_top, output("data_hor_top.vtp") )


![Box Data](./pictures/box_data.png)
Then, we link stratigraphic evidences to the corresponding object in the horizon stack.horizons defined in the horizons stack in each block of the structural model.

In [ ]:
# The model have only one block then every data are in this block
def get_block(impliciter):
    for block in impliciter.structural_model().blocks() :
        return block
    
bot_horizon_id = impliciter.horizons_stack().bottom_horizon()
impliciter.add_horizon_data_points_in_block( 
    get_block(impliciter ), 
    impliciter.horizons_stack().horizon( bot_horizon_id ),
    data_hor_bot, 1 )
impliciter.add_horizon_data_points_in_block( 
    get_block(impliciter ), 
    impliciter.horizons_stack().horizon( 
        impliciter.horizons_stack().above( impliciter.horizons_stack().above( bot_horizon_id ) ) ),
    data_hor_top, 1 )

___
In short, INPUTS from feflow are:
 * Evidences point coordinates
 * Link between each point and its stratigraphic discontinuity (defined in the horizons stack)
___

### Interpolate implicit function
The implicit scalar field is ready to be interpolated in each block.

In [ ]:
# Compute implicit function and create the resulting ImplicitStructuralModel
impliciter_result = impliciter.build(geode_implicit.StructuralModelImplicitationParameters(3))

### Explicit horizons in the structural model
Each horizon in the horizons stack can be explicited individualizing stratigraphic units.

In [ ]:
# Explicit horizons Output
expliciter = geode_implicit.ImplicitStructuralModelHorizonExpliciter( impliciter_result )
expliciter.explicit_all_horizons()
expliciter_result = expliciter.build()
og.save_brep( expliciter_result, output("box_with_horizons.vtm") )

![Box horizons](./pictures/box_horizons.png)

The resulting geometry can be modified moving evidence points.

## Implicit Modeling In A Faulted Model

The input structural model only consider one fault, more complex geometry can be used. A temporary limitation is that faults should cross the volume of interest completly. 

![input structural model](./pictures/mss_in.png)

### Load Faulted Structural Model.. Build Impliciter Tool.

In [ ]:
def horizons_stack():
    horizons_names = [ "hor_bot", "hor_top" ]
    su_names = [ "su_bot", "su_mid", "su_top" ]
    return ogg.horizons_stack_from_name_list_3d( horizons_names, su_names )

# load structural model with two fault blocks.
structural_model = ogg.load_structural_model( input("mss.og_strm") )
#save to vtm for a visualization in paraview
og.save_brep(structural_model,output("mss.vtm"))    

# Create StructuralModelImpliciter with a StructuralModel and
# HorizonsStack
# Other possibility: create with just StructuralModel and modify
# HorizonsStack afterwards through the horizons_stack_builder function.
impliciter = geode_implicit.StructuralModelImpliciter( ogg.load_structural_model( input("mss.og_strm") ), horizons_stack())
# Create and set data to use
impliciter_model = impliciter.structural_model()
impliciter_stack = impliciter.horizons_stack()

### Create data points...

In [ ]:
def create_points_set(points):
    mesh = og.PointSet3D.create()
    mesh_builder = og.PointSetBuilder3D.create(mesh)
    mesh_builder.create_vertices(len(points))
    id=0
    for pt in points:
        mesh_builder.set_point(id,pt)
        id+=1
    return mesh

#create data related to the bottom horizon in the lower fault block
bot_bot_data = create_points_set([
    og.Point3D([ -5000, -3102, -1499 ]), og.Point3D([ -5000, 0, -1499 ]),
    og.Point3D([ -5000, 5000, -1499 ]), og.Point3D([ 0, -3102, -1499 ]),
    og.Point3D([ 0, 0, -1499 ]), og.Point3D([ 0, 5200, -1499 ]),
    og.Point3D([ 7000, -3102, -1499 ]), og.Point3D([ 7000, 0, -1499 ]), 
    og.Point3D([ 7000, 5200, -1499 ]) ])
og.save_point_set3D( bot_bot_data, output("bot_bot_data.vtp") )

#create data related to the bottom horizon in the upper fault block
bot_top_data = create_points_set([
    og.Point3D([ -5000, -3102, 955 ]), og.Point3D([ -5000, 0, 955 ]),
    og.Point3D([ -5000, 5000, 955 ]), og.Point3D([ 2000, -3102, 455 ]),
    og.Point3D([ 2000, 0, 455 ]), og.Point3D([ 2000, 5200, 455 ]) ])
og.save_point_set3D( bot_top_data, output("bot_top_data.vtp") )

#create data related to the top horizon in the lower fault block
top_bot_data = create_points_set([
    og.Point3D([ 9000, -3102, 405 ]), og.Point3D([ 9000, 0, 455 ]),
    og.Point3D([ 9000, 5000, 405 ]), og.Point3D([ 2000, -3102, 1099 ]),
    og.Point3D([ 2000, 0, 1099 ]), og.Point3D([ 2000, 5200, 1099 ]) ])
og.save_point_set3D( top_bot_data, output("top_bot_data.vtp") )

#create data related to the top horizon in the upper fault block
top_top_data = create_points_set([
    og.Point3D([ -5000, -3102, 2800 ]), og.Point3D([ -5000, 0, 2800 ]),
    og.Point3D([ -5000, 5000, 2800 ]), og.Point3D([ 0, -3102, 2800 ]),
    og.Point3D([ 0, 0, 2800 ]), og.Point3D([ 0, 5200, 2800 ]), 
    og.Point3D([ 7000, -3102, 2800 ]), og.Point3D([ 7000, 0, 2800 ]),
    og.Point3D([ 7000, 5200, 2800 ]) ])
og.save_point_set3D( top_top_data, output("top_top_data.vtp") )

### ... to link with the horizons stack.

In [ ]:
# On bottom block in the model
bot_block_id = og.uuid( "00000000-a8b7-4ef1-8000-0000146ebd41" )
bot_block = impliciter_model.block( bot_block_id )
bot_horizon_id = impliciter_stack.bottom_horizon()
impliciter.add_horizon_data_points_in_block( bot_block, impliciter_stack.horizon( bot_horizon_id ), bot_bot_data, 1 )
top_horizon = impliciter_stack.horizon( impliciter_stack.above( impliciter_stack.above( bot_horizon_id ) ) )
impliciter.add_horizon_data_points_in_block( bot_block, top_horizon, bot_top_data, 1 )
# On the top block in the model
top_block_id = og.uuid( "00000000-0dd3-4b37-8000-000043469436" )
top_block = impliciter_model.block( top_block_id )
impliciter.add_horizon_data_points_in_block( top_block, impliciter_stack.horizon( bot_horizon_id ), top_bot_data, 1 )
impliciter.add_horizon_data_points_in_block( top_block, top_horizon, top_top_data, 1 )

### Build scalar field and explicit horizons.

In [ ]:
# Compute implicit function and create the resulting ImplicitStructuralModel
impliciter_result = impliciter.build(geode_implicit.StructuralModelImplicitationParameters(3))
# Explicit horizons Output
expliciter = geode_implicit.ImplicitStructuralModelHorizonExpliciter( impliciter_result )
expliciter.explicit_all_horizons()
expliciter_result = expliciter.build()
og.save_brep( expliciter_result, output("mss_from_implicit_modeling.vtm") )